# Kerasを用いた画像分類入門（４）

##  転移学習とファインチューニング

## Google Colab用の設定

Google Colaboratory で実行する場合には下記のセルを実行してください。

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip /content/gdrive/My\ Drive/GD_share01/noodles.zip -d /content

## ライブラリのインポート

In [ ]:
import tensorflow.keras as keras

## 画像データの準備

 解凍した教師画像からデータを読み込むImageDataGeneratorを作成します。  
 下記の訓練用のImageDataGeneratorの初期化パラメータを設定してください。

In [ ]:
train_dir = './noodles/train/'
validation_dir = './noodles/validation/'
test_dir = './noodles/test/'
image_size = (128, 128)
batch_size = 32
steps_per_epoch = 300 // batch_size
validation_steps = 100 // batch_size
test_steps = 100 // batch_size

augmented_data_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                                       rotation_range=40.0,
                                                                       width_shift_range=0.1,
                                                                       height_shift_range=0.1,
                                                                       brightness_range=[0.8, 1.0],
                                                                       shear_range=20.0,
                                                                       zoom_range=0.1,
                                                                       horizontal_flip=True,
                                                                       vertical_flip=True)

rescaled_data_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)

train_generator = augmented_data_generator.flow_from_directory(train_dir,
                                                           target_size=image_size, 
                                                           batch_size=batch_size)
validation_generator = rescaled_data_generator.flow_from_directory(validation_dir,
                                                                target_size=image_size, 
                                                                batch_size=batch_size)
test_generator = rescaled_data_generator.flow_from_directory(test_dir,
                                                                target_size=image_size, 
                                                                batch_size=batch_size)

## 学習済みVGG16を使った転移学習

###  モデルの構築

学習済みのVGG16ネットワークをロードします。

In [ ]:
conv_layers = keras.applications.vgg16.VGG16(include_top=False, 
                                             weights='imagenet', 
                                             input_shape=(128, 128, 3))

重みを凍結（学習できないように設定）します。

In [ ]:
conv_layers.trainable = False

この学習済みネットワークを組み込んだモデルを作成します。

In [ ]:
conv_layers.summary()

In [ ]:
model = keras.models.Sequential()

model.add(conv_layers)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(2, activation='softmax'))

### モデル構造の表示

In [ ]:
model.summary()

### モデルのコンパイル

モデルをコンパイルします。

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['acc'])

### モデルの訓練

ここでは30エポック、モデルの訓練を行います。

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=30,
                              steps_per_epoch=steps_per_epoch,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

### 学習経過のグラフ化

下記のプログラムで、エポックごとの損失と正解率をグラフ化できます。  
グラフから何が読み取れるでしょうか？

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, label='Train Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.legend()
plt.show()

### ファインチューニング

ファインチューニングを行うために凍結したconv_layersのblock5の層の凍結を解除します。

In [ ]:
conv_layers.trainable = True

for layer in conv_layers.layers:
    if layer.name in  ['block5_conv1', 'block5_conv2', 'block5_conv3']:
        layer.trainable = True
    else:
        layer.trainable = False
    
model.summary()

学習率を下げてコンパイルし直します。

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['acc'])

### モデルの訓練

ここでは30エポック、モデルの訓練を行います。

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=50,
                              steps_per_epoch=steps_per_epoch,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

### 学習経過のグラフ化

下記のプログラムで、エポックごとの損失と正解率をグラフ化できます。  
グラフから何が読み取れるでしょうか？

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, label='Train Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.legend()
plt.show()

### モデルの評価

In [ ]:
evaluation = model.evaluate_generator(test_generator, steps=test_steps)
print(evaluation)

最も性能の良かったモデルを読み込みます。

## モデルによる予測

下記のプログラムで、学習したモデルを使ってテスト用データの分類（予測）を行うことができます。  
予測結果がどのような形式なのかを確認してください。

In [ ]:
predictions = model.predict_generator(test_generator, steps=test_steps)

print(predictions)

## モデルの予測結果を見てみる

次のプログラムを実行すると、テスト用データの１バッチ分についてモデルの予測値と正解を画像つきで確認できます。  
どのような結果になったかを確認してください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

code_to_class = {0: 'ラーメン', 1: 'うどん'}

x, y = next(test_generator)
y_true = np.argmax(y, axis=1)
y_prob = model.predict_on_batch(x)
y_pred = np.argmax(y_prob, axis=1)

for i in range(len(x)):
    plt.imshow(x[i])
    plt.show()
    print('推定： {}({:.5f})'.format(code_to_class[y_pred[i]], y_prob[i, y_pred[i]]))
    print('正解：', code_to_class[y_true[i]])